In [43]:
# Import stuff <3
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [44]:
# Find our letters
group_nr = 2
letters = "ABCDEFGHIJKLMNOPRSTUVWZABCDEFGHIJKLMNOPRSTUVWZ"[group_nr%23:group_nr%23+10]

In [45]:
base = "https://en.wikinews.org/" # For later use

# Create list with urls depending on letters
url_list = []
for l in letters:
    url_list.append('https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from={}'.format(l))

In [46]:
# Create empty lists
all_titles = []
all_links = []
all_dates = []
all_content = []
all_sources = []

# Loops through url list to retrieve titles and links
for url,l in zip(url_list,letters):
    
    # Needed for doing multiple passes
    all_found = False
    page_url = url
    
    # While there are still some articles not found
    while not all_found:
        response = requests.get(page_url)
        soup = BeautifulSoup(response.content, "html.parser")
        container = soup.find(id = "mw-pages")
        
        data = container.find_all('li')
        
        # Add links to our list
        links = [base + link.get('href') for link in container.find_all('a') if link.get('href')[:6] == "/wiki/"]
        
        all_links += links
        
        # Added so it only takes titles with the correct first letter
        titles = [m.text for m in data if m.text[0].upper()==l]
        
        # Append the titles to the running list
        all_titles += titles
        
        # If the last element on the page is not part of our letter we go to the next
        if data[-1].text[0] != l:
            break
            
        # Otherwise we find the "next page" button and follows it
        buttons = soup.findAll('a', href=True)
        for b in buttons:
            try:
                if b.contents[0] == "next page":
                    page_url = base + b["href"]
            except:
                pass

In [47]:
# Loops through links to retrieve data from each articles
for article in all_links[0:9]:
    response = requests.get(article)
    soup = BeautifulSoup(response.content, "html.parser")
    container = soup.find('div', class_ = "mw-parser-output")

    # Date
    date = [d.text for d in container.find_all("strong", class_ = "published")]
    all_dates += date

    # Content
    content = [c.text for c in container.find_all("p")[1:]]
    all_content += content

    # Sources
    sources = [s.text for s in container.find_all("span", class_ = "sourceTemplate")]
    all_sources += sources

In [59]:
# Organizing data
data = {'Title': all_titles, 'Link': all_links, 'Date': all_dates, 'Content': all_content, 'Sources': all_sources}

In [58]:
# Create dataframe from data
df = pd.DataFrame(data)

# Export data as csv and excel
df.to_csv("/Users/gabriellemadsen/Library/CloudStorage/OneDrive-Personligt/Uddannelse/Machine Learning & Datavidenskab/2. semester/DS/Milestone_1/wikinewsdata.csv")
df.to_excel("/Users/gabriellemadsen/Library/CloudStorage/OneDrive-Personligt/Uddannelse/Machine Learning & Datavidenskab/2. semester/DS/Milestone_1/wikinewsdata.xlsx")